In [4]:
import pandas as pd
import arnie
from arnie.utils import *
from arnie.utils import _group_into_non_conflicting_bp

# import csv for pseudoknot predictions

def get_csv(csv_loc):
    df = pd.read_csv(csv_loc)
    return df 

# extract locations for each pseudoknot along with dotbracket structures

def check_if_shapeknots(name, shapeknots_names):
    for program in shapeknots_names: 
        if name == program: 
            return True

def get_info(df):
    starts = df['start'].to_list()
    ends = df['end'].to_list()
    sequences = df['sequence'].to_list()
    dotbrackets = df['struct'].to_list()
    
    return starts, ends, sequences, dotbrackets

# use the below function only for dataframe analysis using shape-directed threshknot csvs

def get_info_from_shapeknots(df):
    starts = (df['start']-1).to_list()
    ends = df['end'].to_list()
    sequences = df['sequence'].to_list()
    dotbrackets = df['threshknot_structure'].to_list()
    
    return starts, ends, sequences, dotbrackets

# import shapeknots data and convert to list

def get_shape_data(filename):
    shape = []
    with open(filename) as f:
        for line in f:
            line = line.strip()
            shape.append(line)
            
    for i in range(len(shape)):
        shape[i] = (-1) if shape[i] == 'nan' else float(shape[i])
        
    return shape

# use Rachael's function to compare shape and dotbracket structure and return ranking

def evaluate_L1_shape_score(s,shape):
    score = 0
    for c,react in zip(s,shape):
        if (c=="." and react>0.25) or (c!="." and react<0.5):
            score += 1
    return score/len(s)

# get locations of pseudoknotted base pairs in a window

def get_groups(dotbracket):
    bp_list = convert_dotbracket_to_bp_list(dotbracket, allow_pseudoknots=True)
    groups = _group_into_non_conflicting_bp(bp_list)
    return groups

def get_pk_bp_locs(groups):
    pk_bp_list = []
    pk_bp_locs = []
    for i, lists in enumerate(groups):
        if i == 0: 
            None
        else: 
            length = len(lists)
            for idx in range(length):
                bp = lists[idx]
                pk_bp_list.append(bp)
                pk_bp_locs.append(bp[0])
                pk_bp_locs.append(bp[1])             
    pk_bp_locs.sort()
    return pk_bp_locs, pk_bp_list

def get_pk_bp_struct(pk_bp_locs, dotbracket):
    pk_bp_struct = []
    for idx in pk_bp_locs:
        bracket = dotbracket[idx]
        pk_bp_struct.append(bracket)
    return pk_bp_struct

# rank PKs based on theoretically thermodynamic stability 

# ranking function operations: 
# add 1 for every additional base pair in a helix
# subtract 1 for every base pair adjacent to a separate helix 

def get_pk_rank(pk_bp_locs, dotbracket):
    pk_rank = 0
    for idx in pk_bp_locs:
        if (idx != 119) and (dotbracket[idx] == dotbracket[idx+1]):
            pk_rank += 0.5   
    for idx in pk_bp_locs: 
        if (idx != 119) and (dotbracket[idx] != dotbracket[idx+1]):
            if dotbracket[idx+1] == '.':
                pk_rank += 0
            else:
                pk_rank -= 1
    for idx in pk_bp_locs: 
        if (idx != 0) and (dotbracket[idx] != dotbracket[idx-1]):
            if dotbracket[idx-1] == '.':
                pk_rank += 0
            else:
                pk_rank -= 1
    return pk_rank

# rank PKs on consensus with other predictions

def get_bp_list(dotbracket):
    bp_list = convert_dotbracket_to_bp_list(dotbracket, allow_pseudoknots=True)
    return bp_list

def compare_bp_lists(bp_list1, bp_list2):
    bp_list_score = 0
    for bp1 in bp_list1: 
        for bp2 in bp_list2: 
            if bp1 == bp2: 
                bp_list_score += 1
        # divide by total number of base pairs in bp_list1 to normalize results
    return bp_list_score/len(bp_list1)

def check_if_same_window(start1, program2_starts):
    for start2 in program2_starts: 
        if start1 == start2: 
            return True

def get_consensus_scores(program1_starts, bp_lists1, program2_starts, bp_lists2):
    scores = []
    for i, start1 in enumerate(program1_starts):
        if check_if_same_window(start1, program2_starts):
            for idx, start2 in enumerate(program2_starts):
                if start1 == start2:
                    bp_list1 = bp_lists1[i]
                    bp_list2 = bp_lists2[idx]
                    bp_list_score = compare_bp_lists(bp_list1, bp_list2)
                    scores.append(bp_list_score)
        elif not check_if_same_window(start1, program2_starts):
            scores.append(0)
    return scores

def get_weighted_consensus(starts_by_program, bp_lists_by_program):
    weighted_avgs = []
    for idx1, program1_starts in enumerate(starts_by_program):
        program1_bp_lists = bp_lists_by_program[idx1]
        program1_consensus_scores = []
        for idx2, program2_starts in enumerate(starts_by_program):
            if idx1 != idx2:
                program2_bp_lists = bp_lists_by_program[idx2]
                consensus_scores = get_consensus_scores(program1_starts, program1_bp_lists, program2_starts, program2_bp_lists)
                program1_consensus_scores.append(consensus_scores)
        
        program1_weighted_avgs = []
        for i in range(len(program1_consensus_scores[0])):
            window = []
            for program in program1_consensus_scores:
                window.append(program[i])
            weighted_avg = sum(window)/len(window)
            program1_weighted_avgs.append(weighted_avg)
            
        weighted_avgs = weighted_avgs + program1_weighted_avgs
    return weighted_avgs

#### this function takes as input a list of dotbracket structures and bp_lists for pseudoknots predicted in the same window
#### returns as output a score for consensus of all to all for each structure

def all_to_all(structs_list, bp_lists):
    scores = []
    for idx, struct in enumerate(structs_list): 
        score = 0
        for i, char in enumerate(struct):
            same_np = 0
            for other_struct in structs_list: 
                if (char == '.') and (char == other_struct[i]):
                    same_np += 1
            if same_np == len(structs_list):
                score += 1
        for bp in bp_lists[idx]:
            same_bp = 0
            for bp_list in bp_lists: 
                for bp2 in bp_list:
                    if bp == bp2:
                        same_bp += 1
            if same_bp == len(bp_lists):
                score += 1
        
        count = 0
        for char in struct:
            if char == '.':
                count += 1
        count += len(bp_lists[idx])
        
        score = score/count
        scores.append(score)
        
    return scores

# create new dataframe with rankings

def get_df(all_programs, starts, ends, sequences, dotbrackets, shape_scores, pk_bp_shape_scores, ranks, weighted_consensus_avgs, weighted_pk_bps_consensus_avgs):
    PK_list = zip(all_programs, starts, ends, sequences, dotbrackets, shape_scores, pk_bp_shape_scores, ranks, weighted_consensus_avgs, weighted_pk_bps_consensus_avgs)
    df = pd.DataFrame(PK_list, columns = ['program', 'start', 'end', 'sequence', 'structure', 'shape_score', 'pk_bp_shape_score', 'rank', 'weighted_consensus_score', 'weighted_pk_bp_consensus_score'])
    ranked_df = df.sort_values('pk_bp_shape_score', ascending=False)
    return ranked_df

# get consensus score for only pk bps
    
# put it all together

def score_pk_overall(names, shapeknots_names, path, shape_file):
    
    all_programs = []
    starts = []
    ends = []
    sequences = []
    dotbrackets = []
    
    starts_by_program = []
    ends_by_program = []
    sequences_by_program = []
    dotbrackets_by_program = []
    bp_lists_by_program = []
    pk_bp_lists_by_program = []
    
    for name in names: 
        df = get_csv(path + name + '.csv')
        
        if check_if_shapeknots(name, shapeknots_names):
            program_starts, program_ends, program_sequences, program_dotbrackets = get_info_from_shapeknots(df)
        elif not check_if_shapeknots(name, shapeknots_names):
            program_starts, program_ends, program_sequences, program_dotbrackets = get_info(df)
        
        for i in range(len(program_starts)):
            all_programs.append(name)
            
        program_bp_lists = []
        for dotbracket in program_dotbrackets: 
            bp_list = get_bp_list(dotbracket)
            program_bp_lists.append(bp_list)
            
        pk_bp_list_by_program = []
        pk_bp_locs_by_program = []
        pk_bp_structs_by_program = []
        for i, struct in enumerate(program_dotbrackets):
            groups = get_groups(struct)
            pk_bp_loc, pk_bp_list = get_pk_bp_locs(groups)
            pk_bp_struct = get_pk_bp_struct(pk_bp_loc, struct)
        
            pk_bp_locs_by_program.append(pk_bp_loc)
            pk_bp_structs_by_program.append(pk_bp_struct)
            pk_bp_list_by_program.append(pk_bp_list)
        
        
        starts = starts + program_starts
        ends = ends + program_ends
        sequences = sequences + program_sequences
        dotbrackets = dotbrackets + program_dotbrackets
        
        starts_by_program.append(program_starts)
        ends_by_program.append(program_ends)
        sequences_by_program.append(program_sequences)
        dotbrackets_by_program.append(program_dotbrackets)
        bp_lists_by_program.append(program_bp_lists)
        pk_bp_lists_by_program.append(pk_bp_list_by_program)
        
    
    # get rough score for consensus with shape data for entire window
    
    full_shape = get_shape_data(shape_file)
    shapes = []
    for i, start in enumerate(starts):
        end = ends[i]
        shape_window = full_shape[start:end]
        shapes.append(shape_window)
    
    shape_scores = []
    for i, struct in enumerate(dotbrackets):
        shape = shapes[i]
        score = evaluate_L1_shape_score(struct, shape)
        shape_scores.append(score)

    # get score for shape consensus with only pk bps

    pk_bp_lists = []
    pk_bp_locs = []
    pk_bp_structs = []
    for i, struct in enumerate(dotbrackets):
        groups = get_groups(struct)
        pk_bp_loc, pk_bp_list = get_pk_bp_locs(groups)
        pk_bp_struct = get_pk_bp_struct(pk_bp_loc, struct)
        
        pk_bp_locs.append(pk_bp_loc)
        pk_bp_structs.append(pk_bp_struct)
        pk_bp_lists.append(pk_bp_list)
        
    pk_bp_shapes = []
    for i, locs in enumerate(pk_bp_locs):
        pk_bp_shapes_window = []
        shape_window = shapes[i]
        for idx in locs:
            shape = shape_window[idx]
            pk_bp_shapes_window.append(shape)
        pk_bp_shapes.append(pk_bp_shapes_window)
        
    pk_bp_shape_scores = []
    for i, struct in enumerate(pk_bp_structs):
        pk_bp_shape_window = pk_bp_shapes[i]
        score = evaluate_L1_shape_score(struct, pk_bp_shape_window)
        pk_bp_shape_scores.append(score)
    
    # get rough ranking for likelihood of PK
        
    ranks = []
    for i, struct in enumerate(dotbrackets): 
        rank = get_pk_rank(pk_bp_locs[i], struct)
        ranks.append(rank)
        
    # get consensus score with other predictions
    
    weighted_consensus_avgs = get_weighted_consensus(starts_by_program, bp_lists_by_program)
    
    # get consensus scores for pk bps only 
    
    weighted_pk_bps_consensus_avgs = get_weighted_consensus(starts_by_program, pk_bp_lists_by_program)
    
    # put it all together into a dataframe
        
    df = get_df(all_programs, starts, ends, sequences, dotbrackets, shape_scores, pk_bp_shape_scores, ranks, weighted_consensus_avgs, weighted_pk_bps_consensus_avgs)
    return df

In [6]:
names = ['incarnato_invitro', 'knotty', 'threshknot', 'zhang_invivo', 'incarnato_invivo', 'pknots', 'zhang_invitro', 'pyle', 'spotrna']
shapeknots_names = ['incarnato_invitro', 'zhang_invivo', 'incarnato_invivo', 'zhang_invitro', 'pyle']
path = '/home/gnye8/Desktop/PK_research/pipeline_results/direct_output/'
shape = '/home/gnye8/Desktop/PK_research/SSRP_work/shape_data/incarnato_invivo_reactivity-Copy1.csv'


test_df = score_pk_overall(names, shapeknots_names, path, shape)

In [7]:
test_df.to_csv('/home/gnye8/Desktop/PK_research/pipeline_results/analysis_output/first_weighted_consensus.csv')

In [22]:
from Bio import SeqIO

def get_seq(seq_filename):
    record = SeqIO.read(seq_filename, "fasta")
    return str(record.seq)

starts = [801]
ends = [920]
sequence = 'GAGGGGCATACACTCGCTATGTCGATAACAACTTCTGTGGCCCTGATGGCTACCCTCTTGAGTGCATTAAAGACCTTCTAGCACGTGCTGGTAAAGCTTCATGCACTTTGTCCGAACAAC'
full_shape = get_shape_data('/home/gnye8/Desktop/PK_research/SSRP_work/shape_data/incarnato_invivo_reactivity-Copy1.csv')
full_sars = get_seq('/home/gnye8/Desktop/PK_research/SSRP_work/fasta_files/SARS_CoV2_reference.fasta')

test_seq = full_sars[start-1:end]

print(test_seq)

shapes = []
for i, start in enumerate(starts):
    end = ends[i]
    shape_window = full_shape[start-1:end]
    shapes.append(shape_window)
    
print(len(sequence))
print(len(shapes[0]))

GAGGGGCATACACTCGCTATGTCGATAACAACTTCTGTGGCCCTGATGGCTACCCTCTTGAGTGCATTAAAGACCTTCTAGCACGTGCTGGTAAAGCTTCATGCACTTTGTCCGAACAAC
120
120


In [36]:
incarnato_invivo_vs_threshknot.to_csv('/home/gnye8/Desktop/PK_research/pipeline_results/shape_directed_threshknot/incarnato_invivo_vs_threshknot.csv')

In [56]:
names = ['incarnato_invitro', 'incarnato_invivo', 'knotty', 'pknots', 'threshknot', 'zhang_invivo']
shapeknots_names = ['incarnato_invitro', 'incarnato_invivo', 'zhang_invivo']

def is_name_shapeknots(name, shapeknots_names):
    if (name == shapeknots_names[0]) or (name == shapeknots_names[1]) or (name == shapeknots_names[2]):
        return True

def get_analysis_csvs(names, shapeknots_names):
    
    for i, name1 in enumerate(names):
        for name2 in names:
            if name1 == name2:
                None
            else:
                if (is_name_shapeknots(name1, shapeknots_names)) and (is_name_shapeknots(name2, shapeknots_names)):
                    analysis_csv = score_pk_overall('/home/gnye8/Desktop/PK_research/pipeline_results/direct_output/{}.csv'.format(name1), '/home/gnye8/Desktop/PK_research/SSRP_work/shape_data/incarnato_invivo_reactivity-Copy1.csv', '/home/gnye8/Desktop/PK_research/pipeline_results/direct_output/{}.csv'.format(name2), csv1_shapeknots=True, csv2_shapeknots=True)
                elif is_name_shapeknots(name1, shapeknots_names):
                    analysis_csv = score_pk_overall('/home/gnye8/Desktop/PK_research/pipeline_results/direct_output/{}.csv'.format(name1), '/home/gnye8/Desktop/PK_research/SSRP_work/shape_data/incarnato_invivo_reactivity-Copy1.csv', '/home/gnye8/Desktop/PK_research/pipeline_results/direct_output/{}.csv'.format(name2), csv1_shapeknots=True)
                elif is_name_shapeknots(name2, shapeknots_names):
                    analysis_csv = score_pk_overall('/home/gnye8/Desktop/PK_research/pipeline_results/direct_output/{}.csv'.format(name1), '/home/gnye8/Desktop/PK_research/SSRP_work/shape_data/incarnato_invivo_reactivity-Copy1.csv', '/home/gnye8/Desktop/PK_research/pipeline_results/direct_output/{}.csv'.format(name2), csv2_shapeknots=True)
                else:
                    analysis_csv = score_pk_overall('/home/gnye8/Desktop/PK_research/pipeline_results/direct_output/{}.csv'.format(name1), '/home/gnye8/Desktop/PK_research/SSRP_work/shape_data/incarnato_invivo_reactivity-Copy1.csv', '/home/gnye8/Desktop/PK_research/pipeline_results/direct_output/{}.csv'.format(name2))
                analysis_csv.to_csv('/home/gnye8/Desktop/PK_research/pipeline_results/analysis_output/{}_vs_{}_analysis_scores.csv'.format(name1, name2))

In [58]:
analysis_csvs = get_analysis_csvs(names, shapeknots_names)

,start,end,sequence,structure,shape_score,pk_bp_shape_score,rank,consensus_score,pk_bp_consensus_score
3,5240,5360,CACAAGTTAATGGTTTAACTTCTATTAAATGGGCAGATAACAACTG...,...(((((((....)))))))..........((((((((((....)...,0.658333,1.000000,3.0,0.000,0.000000
4,6520,6640,ATTACAGAAGAGGTTGGCCACACAGATCTAATGGCTGCTTATGTAG...,...............(((((...........))))).........(...,0.608333,1.000000,8.0,0.000,0.000000
5,6560,6680,ATGTAGACAATTCTAGTCTTACTATTAAGAAACCTAATGAATTATC...,.....(((.......)))............(((((((............,0.575000,1.000000,8.0,0.000,0.000000
11,14160,14280,TTAATGCCTATATTAACCTTGACCAGGGCTTTAACTGCAGAGTCAC...,.........................((((........(((.((((....,0.750000,0.954545,7.0,0.000,0.000000
14,20000,20120,TGATGGTCAAGTAGACTTATTTAGAAATGCCCGTAATGGTGTTCTT...,...(........((((.................................,0.425000,0.954545,6.0,0.000,0.000000
12,18080,18200,AATCACTGGGTTACATCCTACACAGGCACCTACACACCTCAGTGTT...,.......(((((...((((.................(.(((((.((...,0.575000,0.923077,7.0,0.000,0.000000
0,480,600,ACGTTCGGATGCTCGAACTGCACCTCATGGTCATGTTATGGTTGAG...,...(((([[[[[.))))((((............................,0.800000,0.888889,4.0,0.000,0.000000
8,9200,9320,TTGATTCTGAGTACTGTAGGCACGGCACTTGTGAAAGATCAGAAGC...,.........((((((.....................[[[..........,0.675000,0.888889,7.0,0.000,0.000000
16,24400,24520,AAACTTCAAGATGTGGTCAACCAAAATGCACAAGCTTTAAACACGC...,.....(((((((((((......((((...((((((.........))...,0.625000,0.875000,3.0,0.000,0.000000
9,10240,10360,AATCATAATTTCTTGGTACAGGCTGGTAATGTTCAACTCAGGGTTA...,..........((((((....(((((...(((([[[[[[...[[......,0.675000,0.875000,7.0,0.000,0.000000
